# Out Of Memory(OOM)
## OOM이 해결하기 어려운 이유들
- 왜 발생했는지 알기 어렵다.
- 어디서 발생했는지 알기 어렵다.
- Error backtracking이 이상한데로 간다.
- 메모리의 이전 상황의 파악이 어렵다.

제일 간단한 방법 : batch size 줄이고 gpu clean 후 다시 돌리기

### 그 외에 발생할 수 있는 문제가 있을 수 있다..

# GRPUUtil 사용하기
- nvidia-smi 처럼 GPU의 상태를 보여주는 모듈이다.
- Colab은 환경에서 GPU 상태를 보여주기 편하다.
- iter마다 메모리가 늘어나는지 확인!


In [4]:
!pip install GPUtil

  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=3453378f7f0d2f8b2aaa26856cc6fd24076f808dc3f115372f6fe6a9179f414f
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [ ]:
import GPUtil
GPUtil.showUtilization()

# torch.cuda.empty_cache() 써보기
- 사용되지 않는 GPU상 cache를 정리한다.
- 가용 메모리를 확보한다.
- del과는 구분이 필요하다.
- reset 대신 쓰기 좋은 함수이다.

## trainning loop에 tensor로 축적 되는 변수는 확인할 것
- tensor로 처리된 변수는 GPU상에 메모리 사용
- 해당 변수 loop 안에 연산에 있을 때 GPU에 computational graph를 생성(메모리 잠식)

## del 명령어를 적절히 사용하기
- 필요가 없어진 변수는 적절한 삭제가 필요하다.
- python의 메모리 배치 특성상 loop이 끝나도 메모리를 차지한다.

In [ ]:
# Example
'''
for i in range(5):
    intermediate = f(inputresult += g(intermediate)
output = h(result)
return output

del intermediate
'''


In [ ]:
# 학습시 OOM이 발생했다면 batch 사이즈를 1로 해서 실험해보기
'''
oom = False
try:
    run_model(batch_size)
except RuntimeError: # out of memory
    oom = True

if oom:
    for _ in range(batch_size):
        run_model(1)
'''

## torch.no_grad() 사용하기
- inference 시점에서는 torch.no_grad() 구문을 사용한다.
- backward pass로 인해 쌓이는 메모리에서 자유롭다.

In [ ]:
# Example

with torch.no_grad():
    for data, target in test_loader:
        output = network(data)
        test_loss += F.nll_loss(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()

# 예상치 못한 에러 메세지
- OOM 말고도 유사한 에러들이 발생한다.
- CUDNN_STATUS_NOT_INIT 이나 device-side-assert 등
- 해당 에러도 cuda와 관련하여 OOM의 일종으로 생각될 수 있으며, 적절한 코드 처리가 필요하다.

# 그 외
- colab에서 너무 큰 사이즈는 실행하지 말 것(linear, CNN, LSTM)
- CNN의 대부분의 에러는 크기가 안 맞아서 생기는 경우(torchsummary 등으로 사이즈를 맞출 것)
- tensor의 float precision을 16bit로 줄일 수도 이다.